In [1]:
import scipy.io as spio
import pandas as pd

In [5]:
def impmat(fname = 'M_processed.mat', writ = True):
    ''' import matlab crap, and turn it to pickles (or return panda df)'''
    mat = spio.loadmat(fname, squeeze_me=True)
    M = mat['M'] # data comes in as a bunch of TRASH, gotta make it PANDA
    #print(mat["__header__"])
    #print(mat["__version__"])
    #print(mat["__globals__"])
    head = ['time','ndc1','ndc2','ndc3','Trade_Partner_Name',
    'Distribution_Center_State','NDC','Distribution_Center_ID_(IC)',
    'Distribution_Center_Zip','Eff_Inv_(EU)','Eff_Inv_(PU)',
    'Qty_Ord_(EU)','Qty_Ord_(PU)']

    data = pd.DataFrame(M, columns=head)
    data["time"] = pd.to_datetime(data["time"], format='%Y%m%d', errors='coerce')

    if writ: # h5 allows your variable to be external
        dt = pd.HDFStore("drugdata.h5") # don't need to import/export! warning, though: huge
        dt['dat'] = data #
    return(data)


In [65]:
def test_hd5(p = 0, q = 0):
    '''test data and run answers to intro quiz'''
    dt = pd.HDFStore("drugdata.h5")["dat"]

    header = dt.columns.tolist()
    # thanks @brock
    def q1(df):
        return(df.Trade_Partner_Name.unique())
    
    def q2(df):
        q2 = df.groupby('Trade_Partner_Name')['Distribution_Center_ID_(IC)'].nunique()
        q2max = q2.max()
        return(q2[q2 == q2max])
    
    def q3(df):
        q3df = df.loc[df["time"].dt.year == 2011]
        q3TotalSales = q3df.groupby('NDC')['Qty_Ord_(PU)'].sum()
        #print(q3TotalSales)
        q3sorted = q3TotalSales.sort_values(ascending = False).head()
        return(q3sorted)
    
    def q4(df):
        q4 = df['NDC'].value_counts()
        NDCLessThan60 = q4[q4 < 60]
        if (NDCLessThan60.size == 0):
            return(None)
        else:
            return(NDCLessThan60.size)
        
    def q5(df):
        q5 =  df.groupby('NDC')['Qty_Ord_(PU)'].std()
        q5max = q5.max()
        NDCHighestVariance = q5[q5 == q5max]
        return(NDCHighestVariance)
    
    def q6(df):
        q6 = df.groupby('NDC')['Qty_Ord_(PU)'].nunique()
        q6ZeroDemand = q6[q6 == 0]
        if (q6ZeroDemand.size == 0):
            return(None)
        else:
            return(q6ZeroDemand.size)
    
    if p:
        for col in header:
            print(dt[col].head())
    if q:
        answers = [q1(dt), q2(dt), q3(dt), q4(dt), q5(dt), q6(dt)]
        for i, ans in enumerate(answers):
            try:
                print('Question %d'%(i+1),  ans)
            except:
                print('Question %d'%(i+1) + str(ans))

In [66]:
#d = impmat()
test_hd5(q=1)

Question 1 [  4.  20.   2.   3.  21.  15.  12.   9.   5.  14.   0.  22.  26.  27.   8.
  24.  10.  13.  11.  16.   7.  19.  23.  17.  25.   6.   1.  18.]
Question 2 Trade_Partner_Name
20.0    50
Name: Distribution_Center_ID_(IC), dtype: int64
Question 3 NDC
55.0     7.610247e+06
109.0    6.056820e+06
154.0    2.019386e+06
25.0     1.419782e+06
7.0      1.248716e+06
Name: Qty_Ord_(PU), dtype: float64
Question 4 None
Question 5 NDC
55.0    8826.568486
Name: Qty_Ord_(PU), dtype: float64
Question 6 None


Question 1: [  4.  20.   2.   3.  21.  15.  12.   9.   5.  14.   0.  22.  26.  27.   8.
  24.  10.  13.  11.  16.   7.  19.  23.  17.  25.   6.   1.  18.]


In [26]:

#print(q2)

Trade_Partner_Name
20.0    50
Name: Distribution_Center_ID_(IC), dtype: int64
